# EDA Análise de RH

### 1. Manipulação

#### 1.1 Imports

In [1]:
import pandas as pd
import sqlite3

#### 1.2 Coleta

In [2]:
df = pd.read_csv('HR_Analytics.csv')
df.head(1)

,EmpID,Age,AgeGroup,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,RM297,18,18-25,Yes,Travel_Rarely,230,Research & Development,3,3,Life Sciences,...,3,80,0,0,2,3,0,0,0,0.0


#### 1.3 DataFrame para SQLite

In [3]:
# Conectando a um DataBase no sql
cnn = sqlite3.connect('EDA_HR.db')

# Movendo o dataframe para o SQL
df.to_sql('hr_analytics', cnn)

1480

In [4]:
# Carregando a extenção sql e o banco de dados
%load_ext sql 
%sql sqlite:///EDA_HR.db

### 2. Análise dos Dados

#### 2.1 Qual é a idade média dos funcionários ?

In [5]:
%%sql 

select 
    ROUND(AVG(Age)) as [Idade Média]
from hr_analytics

 * sqlite:///EDA_HR.db
Done.


Idade Média
37.0


#### 2.2 Quantos funcionários por Faixa Etária ?

In [6]:
%%sql

select
    AgeGroup,
    COUNT(*) as [Qtd de Funcionários]
from hr_analytics
group by AgeGroup
order by COUNT(*) desc

 * sqlite:///EDA_HR.db
Done.


AgeGroup,Qtd de Funcionários
26-35,611
36-45,471
46-55,228
18-25,123
55+,47


#### 2.3 Qual é a média de DailyRate (taxa diária) dos funcionários ?

Média de todos os valores

In [7]:
%%sql 

select 
    ROUND(AVG(DailyRate)) as [Média de DailyRate]
from hr_analytics

 * sqlite:///EDA_HR.db
Done.


Média de DailyRate
801.0


Média por faixa etária

In [8]:
%%sql

select
    AgeGroup,
    ROUND(AVG(DailyRate)) as [Média de DailyRate]
from hr_analytics
group by AgeGroup
order by ROUND(AVG(DailyRate)) desc

 * sqlite:///EDA_HR.db
Done.


AgeGroup,Média de DailyRate
46-55,823.0
55+,808.0
26-35,800.0
36-45,799.0
18-25,773.0


Média por Distancia de Casa ao Trabalho

In [9]:
%%sql

select
    CASE
        WHEN DistanceFromHome <= 5 THEN "1-5"
        WHEN DistanceFromHome > 5 and DistanceFromHome <= 10 THEN "6-10"
        WHEN DistanceFromHome > 10 and DistanceFromHome <= 15 THEN "11-16"
        WHEN DistanceFromHome > 15 and DistanceFromHome <= 20 THEN "16-20"
        WHEN DistanceFromHome > 20 and DistanceFromHome <= 25 THEN "21-25"
        ELSE "25+"
    END as [Distancia de Casa],
    ROUND(AVG(DailyRate)) as [Média de DailyRate]
from hr_analytics
group by [Distancia de Casa]
order by ROUND(AVG(DailyRate)) desc

 * sqlite:///EDA_HR.db
Done.


Distancia de Casa,Média de DailyRate
16-20,852.0
11-16,852.0
1-5,808.0
25+,791.0
6-10,776.0
21-25,755.0


#### 2.4 Qual é o departamento que possui o maior número de funcionários ?

In [10]:
%%sql

select 
    Department,
    COUNT(*)
from hr_analytics
group by Department
order by COUNT(*) desc

 * sqlite:///EDA_HR.db
Done.


Department,COUNT(*)
Research & Development,967
Sales,450
Human Resources,63


#### 2.5 Qual é o nível médio de satisfação no trabalho dos funcionários ?

In [11]:
%%sql

select 
    ROUND(AVG(JobSatisfaction)) as [Nivel Médio de Satisfação]
from hr_analytics

 * sqlite:///EDA_HR.db
Done.


Nivel Médio de Satisfação
3.0


Quantidade de Funcionários por nível de satisfação

In [12]:
%%sql

select 
    JobSatisfaction,
    COUNT(*)
from hr_analytics
group by JobSatisfaction
order by JobSatisfaction desc

 * sqlite:///EDA_HR.db
Done.


JobSatisfaction,COUNT(*)
4,461
3,444
2,282
1,293


Nível Médio de Satisfação por Distancia de Casa ao Trabalho 

In [13]:
%%sql 

select 
    CASE
        WHEN DistanceFromHome <= 5 THEN "1-5"
        WHEN DistanceFromHome > 5 and DistanceFromHome <= 10 THEN "6-10"
        WHEN DistanceFromHome > 10 and DistanceFromHome <= 15 THEN "11-16"
        WHEN DistanceFromHome > 15 and DistanceFromHome <= 20 THEN "16-20"
        WHEN DistanceFromHome > 20 and DistanceFromHome <= 25 THEN "21-25"
        ELSE "25+"
    END as [Distancia de Casa],
    ROUND(AVG(JobSatisfaction)) as [Nivel Médio de Satisfação]
from hr_analytics
group by [Distancia de Casa]
order by ROUND(AVG(JobSatisfaction)) desc

 * sqlite:///EDA_HR.db
Done.


Distancia de Casa,Nivel Médio de Satisfação
6-10,3.0
25+,3.0
21-25,3.0
16-20,3.0
11-16,3.0
1-5,3.0


#### 2.6 Qual é a distância média da casa dos funcionários para o trabalho ?

In [14]:
%%sql

select 
    ROUND(AVG(DistanceFromHome)) as [Distancia Média]
from hr_analytics

 * sqlite:///EDA_HR.db
Done.


Distancia Média
9.0


#### 2.7 Qual é a média de anos que os funcionários estão atuando no mesmo cargo ?

In [15]:
%%sql

select
   ROUND(AVG(YearsInCurrentRole)) as [Anos no Mesmo Cargo]
from hr_analytics

 * sqlite:///EDA_HR.db
Done.


Anos no Mesmo Cargo
4.0


#### 2.8 Quais são os diferentes níveis de educação e quantos funcionários possuem cada nível ?

In [16]:
%%sql

select 
    Education,
    COUNT(*) as [Qtd de Funcionarios]
from hr_analytics
group by Education
order by COUNT(*) desc

 * sqlite:///EDA_HR.db
Done.


Education,Qtd de Funcionarios
3,578
4,399
2,283
1,172
5,48
